In [ ]:
import pandas as pd 
import pickle
import datetime
import re
import networkx as nx
import numpy as np

# Atrial fibrillation (AF) patients on warfarin

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/first_mention_warfarin.pickle', 'rb') as f:
    warfarin = pickle.load(f)

In [ ]:
len(warfarin)

In [ ]:
df = pd.DataFrame(list(warfarin.items()), columns=['client_idcode', 'first_mention_warfarin']) # This code converts a dictionary (warfarin) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_warfarin'
df.shape

In [ ]:
df.head()

In [ ]:
ids = set(df['client_idcode'])
len(ids)

# AF patients on warfarin with history of ischemic stroke & transient ischemic attack (TIA)

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')

In [ ]:
inclusion_criteria = {
    'ischemic_stroke' : ['422504002'],
    'transient_ischemic_attack': ['266257000']
                     }
ic = expand_codes(onto,inclusion_criteria)

In [ ]:
%%time 
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
def dates_ex(pt_data, codes):
    dates = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:
            if dates == None:
                dates = pt_data[x]
            else: 
                dates = dates.union(pt_data[x])
    return dates

rows = []
for pt in ids:
    pt_data = pt2cui_pos_dates.get(pt, {})
    row = {'client_idcode': pt, 'first_mention_warfarin': warfarin[pt]} 
    for concept, codes in ic.items():
        dates = dates_ex(pt_data, codes)
        if dates == None:
            row[f"{concept}_date"] = np.nan
        else:
            row[f"{concept}_date"] = dates
    rows.append(row)

In [ ]:
df = pd.DataFrame(rows)
df.shape

In [ ]:
df.head()

In [ ]:
df.dropna(subset=['ischemic_stroke_date', 'transient_ischemic_attack_date'], how='all', inplace = True)

In [ ]:
df.head()

In [ ]:
row = df.loc[df['client_idcode'] == 'M431247']

In [ ]:
specific_client_idcode = 'M431247'

In [ ]:
if not row.empty:
    # Access the set within the 'Set_Column' and print it
    specific_set = row.iloc[0]['transient_ischemic_attack_date']
    print(f"Set for ID {specific_client_idcode}: {specific_set}")
else:
    print(f"No matching ID found for {specific_client_idcode}")

In [ ]:
df['client_idcode'].nunique()

In [ ]:
%%time
df = df[['client_idcode']].join((df[i].explode() for i in df.iloc[:,1:]))
df.shape

In [ ]:
df.head(6)

In [ ]:
for k in ic.keys():
    df[f'{k}_date'] = pd.to_datetime(df[f'{k}_date']).dt.date

In [ ]:
df.head()

In [ ]:
for k in ic.keys():
    df[f'{k}_delta'] = pd.to_datetime(df['first_mention_warfarin']) - pd.to_datetime(df[f'{k}_date'])

In [ ]:
df.head()

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
for k in ic.keys():
    t1 = df[f'{k}_delta'] >= min_time 
    t2 = df[f'{k}_delta'] <= max_time
    in_window = t1 & t2 
    df[f'{k}_in_window'] = in_window

In [ ]:
df.head()

In [ ]:
ischemic_stroke = df.loc[df['ischemic_stroke_in_window'] == True]
ischemic_stroke = ischemic_stroke[['client_idcode', 'ischemic_stroke_in_window']]
ischemic_stroke.shape

In [ ]:
ischemic_stroke.head()

In [ ]:
ischemic_stroke = set(ischemic_stroke['client_idcode'])
len(ischemic_stroke)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/ischemic_stroke.pickle', 'wb') as f:
    pickle.dump(ischemic_stroke,f)

In [ ]:
transient_ischemic_attack = df.loc[df['transient_ischemic_attack_in_window'] == True]
transient_ischemic_attack = transient_ischemic_attack[['client_idcode', 'transient_ischemic_attack_in_window']]
transient_ischemic_attack.shape

In [ ]:
transient_ischemic_attack.head()

In [ ]:
transient_ischemic_attack = set(transient_ischemic_attack['client_idcode'])
len(transient_ischemic_attack)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/transient_ischemic_attack.pickle', 'wb') as f:
    pickle.dump(transient_ischemic_attack,f)